Install Dependencies

In [1]:
# Install core agentic framework libraries
# langchain       -> LLM orchestration
# langgraph       -> Agentic workflow graph
# langchain-community -> HuggingFace integrations
!pip install -q langchain langgraph langchain-community

# Install LLM acceleration & model libraries
# transformers -> Hugging Face models
# accelerate   -> Hardware optimization
# bitsandbytes -> 4-bit quantization for low-memory GPUs
!pip install -q transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.3 MB/s eta 0:00:00


Import Required Libraries

In [2]:
# Torch is used for model data types and GPU handling
import torch

# TypedDict is used to strictly define shared agent state
from typing import TypedDict

# Hugging Face utilities for loading tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# LangChain wrapper for Hugging Face pipelines
from langchain_community.llms import HuggingFacePipeline

# LangGraph classes for building agent workflows
from langgraph.graph import StateGraph, END


Load Open-Source LLM (Mistral-7B Instruct)

In [3]:
# Open-source instruction-tuned model
# Chosen because it is free, powerful, and works well with reasoning tasks
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

try:
    # Load tokenizer for converting text to tokens
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

    # Set padding token to EOS to avoid generation errors
    tokenizer.pad_token = tokenizer.eos_token

    # Load model in 4-bit mode to fit in free Colab GPU memory
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",          # Automatically map model to GPU/CPU
        load_in_4bit=True,          # Memory-efficient loading
        torch_dtype=torch.float16   # Faster computation on GPU
    )

except Exception as error:
    raise RuntimeError(f"Model loading failed: {error}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Create Text Generation Pipeline

In [4]:
# Create Hugging Face text generation pipeline
# This defines how the model generates responses
text_generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=900,           # Enough for full advisory report
    temperature=0.2,              # Lower value = more deterministic output
    do_sample=True,               # Enable controlled randomness
    return_full_text=False,       # Avoid repeating prompt in output
    eos_token_id=tokenizer.eos_token_id
)

# Wrap the pipeline so LangChain can use it
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


Device set to use cuda:0
/tmp/ipython-input-2062880922.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


Define Shared State (Used by All Agents)

In [5]:
class AdvisorState(TypedDict, total=False):
    """
    Shared state passed between agents.

    Fields:
        risk_profile (str): Investor's risk tolerance
        investment_horizon (str): Time horizon description
        market_analysis (str): Market insights
        short_term_strategy (str): Short-term investment plan
        long_term_strategy (str): Long-term investment plan
        risk_assessment (str): Risk & diversification analysis
        final_report (str): Final advisory report

    Example Input:
        {
            "risk_profile": "Moderate",
            "investment_horizon": "Short and Long Term"
        }
    """
    risk_profile: str
    investment_horizon: str
    market_analysis: str
    short_term_strategy: str
    long_term_strategy: str
    risk_assessment: str
    final_report: str


Define Agents

Market Analysis Agent

In [6]:
def market_analysis_agent(state: AdvisorState) -> AdvisorState:
    """
    Analyzes current market conditions.

    Input:
        state["risk_profile"] -> str

    Output:
        Adds "market_analysis" to state

    Example:
        Input:  risk_profile = "Moderate"
        Output: market_analysis = "Market is moderately volatile..."
    """
    try:
        prompt = f"""
        You are a financial analyst.
        Analyze current market conditions for a {state['risk_profile']} investor.
        """

        # Generate analysis using LLM
        analysis = llm.invoke(prompt)

        # Store result in shared state
        state["market_analysis"] = analysis
        return state

    except KeyError:
        raise ValueError("Missing 'risk_profile' in state.")


Short-Term Investment Agent

In [7]:
def short_term_investment_agent(state: AdvisorState) -> AdvisorState:
    """
    Generates short-term investment strategies.

    Focus:
        Liquidity, capital protection, low volatility.

    Output:
        Adds "short_term_strategy" to state
    """
    prompt = f"""
    You are a financial advisor.

    Market context:
    {state.get("market_analysis", "")}

    Investor risk level: {state.get("risk_profile")}

    Suggest short-term investment strategies.
    """

    state["short_term_strategy"] = llm.invoke(prompt)
    return state


Long-Term Investment Agent

In [8]:
def long_term_investment_agent(state: AdvisorState) -> AdvisorState:
    """
    Generates long-term growth-oriented investment strategies.

    Focus:
        Compounding, diversification, wealth creation.

    Output:
        Adds "long_term_strategy" to state
    """
    prompt = f"""
    You are a long-term investment advisor.

    Market context:
    {state.get("market_analysis", "")}

    Investor risk level: {state.get("risk_profile")}

    Recommend long-term investment options.
    """

    state["long_term_strategy"] = llm.invoke(prompt)
    return state


Risk Analysis Agent

In [9]:
def risk_analysis_agent(state: AdvisorState) -> AdvisorState:
    """
    Evaluates risk exposure and diversification.

    Output:
        Adds "risk_assessment" to state
    """
    prompt = f"""
    You are a risk management expert.

    Short-Term Strategy:
    {state.get("short_term_strategy", "")}

    Long-Term Strategy:
    {state.get("long_term_strategy", "")}

    Analyze overall risk and diversification.
    """

    state["risk_assessment"] = llm.invoke(prompt)
    return state


Final Report Compilation Agent

In [10]:
def final_report_agent(state: AdvisorState) -> AdvisorState:
    """
    Compiles a structured professional investment advisory report.

    Output:
        Adds "final_report" to state
    """
    prompt = f"""
    Create a concise professional investment advisory report.

    Market Analysis:
    {state.get("market_analysis", "")}

    Short-Term Strategy:
    {state.get("short_term_strategy", "")}

    Long-Term Strategy:
    {state.get("long_term_strategy", "")}

    Risk Assessment:
    {state.get("risk_assessment", "")}

    Format:
    Executive Summary
    Key Market Insights
    Short-Term Recommendations
    Long-Term Recommendations
    Risk & Diversification Notes
    """

    state["final_report"] = llm.invoke(prompt).strip()
    return state


Build LangGraph Orchestrator

In [11]:
# Create graph with shared state schema
advisor_graph = StateGraph(AdvisorState)

# Register each agent as a node
advisor_graph.add_node("market_analysis", market_analysis_agent)
advisor_graph.add_node("short_term", short_term_investment_agent)
advisor_graph.add_node("long_term", long_term_investment_agent)
advisor_graph.add_node("risk_analysis", risk_analysis_agent)
advisor_graph.add_node("final_report", final_report_agent)

# Define execution order
advisor_graph.set_entry_point("market_analysis")
advisor_graph.add_edge("market_analysis", "short_term")
advisor_graph.add_edge("short_term", "long_term")
advisor_graph.add_edge("long_term", "risk_analysis")
advisor_graph.add_edge("risk_analysis", "final_report")
advisor_graph.add_edge("final_report", END)

# Compile graph
advisor_graph = advisor_graph.compile()


Execute the Agentic Workflow

In [12]:
# Initial input provided by user
initial_state = {
    "risk_profile": "Moderate",
    "investment_horizon": "Short and Long Term"
}

# Execute agent workflow
final_state = advisor_graph.invoke(initial_state)

# Display final output
print("📊 FINAL INVESTMENT ADVISORY REPORT\n")
print(final_state["final_report"])


📊 FINAL INVESTMENT ADVISORY REPORT

Executive Summary:
The current market conditions present both opportunities and challenges for moderate investors. The global economy is showing signs of recovery from the COVID-19 pandemic, but there are also challenges such as inflation and uncertainty surrounding the Omicron variant of COVID-19. In this report, we provide investment recommendations for moderate investors based on their investment objectives, risk tolerance, and market conditions.

Key Market Insights:
The US economy is showing strong growth, but there are challenges such as inflation and uncertainty surrounding the Omicron variant of COVID-19. The Consumer Price Index (CPI) in the US has risen by 6.2% in October 2021, the highest annual increase since 1990. The Federal Reserve has signaled that it may begin to taper its asset purchase program, which could lead to higher interest rates and negatively impact bond investments.

Short-Term Recommendations:
A moderate investor may cons